In [1]:
import pathlib
import sys
import time

sys.path.append("../featurization")
from data_writer import organize_featurization_data
from loading_classes import Featurization, ImageSetLoader, ObjectLoader
from two_object_loading_classes import (
    ColocalizationFeaturization,
    ColocalizationTwoObject_Loader,
)

In [2]:
# begin profiling timer
start = time.time()

### Set the path to the images 

In [3]:
image_set_path = pathlib.Path("../../data/NF0014/cellprofiler/C4-2/")

### set the channel mapping dictionary

In [4]:
channel_mapping = {
    "nuclei": "405",
    "AGP": "488",
    "ER": "555",
    "Mito": "640",
    "BF": "TRANS",
    "nuclei_mask": "nuclei_",
    "cell_mask": "cell_",
    "cytoplasm_mask": "cytoplasm_",
    "organoid_mask": "organoid_",
}

### Initialize the image set loader

In [5]:
image_set_loader = ImageSetLoader(
    image_set_path=image_set_path,
    spacing=(1, 0.1, 0.1),
    channel_mapping=channel_mapping,
)

### Loop through the image set

In [6]:
# get the total number of iterations
iterations = 0
arg_list = []
for compartment in image_set_loader.compartments:
    for channel in image_set_loader.image_names:
        for unique_index in image_set_loader.unique_compartment_objects[compartment]:
            iterations += 1
            arg_list.append((compartment, channel, unique_index))

In [ ]:
progress = 0
for compartment in image_set_loader.compartments:
    for channel in image_set_loader.image_names:
        for unique_index in image_set_loader.unique_compartment_objects[compartment]:
            # initialize the object loader to access the objects in the image
            objects_loader = ObjectLoader(
                image=image_set_loader.get_image(channel),
                label_image=image_set_loader.get_image(compartment),
                channel_name=channel,
                compartment_name=compartment,
                label_index=4,
            )
            # initialize the featurization class to access the features
            # This enables the measurement of the features
            featurization = Featurization(
                image_set_loader=image_set_loader,
                object_loader=objects_loader,
                neighbors_distance_threshold=10,
            )
            # return the features into a function to save the data
            organize_featurization_data(
                features=featurization.get_features(),
                channel=channel,
                compartment=compartment,
                label_index=unique_index,
                image_set_name=image_set_loader.image_set_name,
                return_df=False,
                output=True,
                output_path=pathlib.Path(
                    f"../profiles_features/{image_set_loader.image_set_name}_{channel}_{compartment}_{unique_index}.parquet"
                ),
            )
            progress += 1
            print((progress / iterations) * 100, "% Complete")

INFO:root:Calculated Area Size Shape features for cell_mask 4


## Run Colocalization

In [ ]:
# get all combinations of channels, compartments, and objects
colocalization_channel_pairs = []

for channel1 in image_set_loader.image_names:
    for channel2 in image_set_loader.image_names:
        if channel1 != channel2:
            colocalization_channel_pairs.append((channel1, channel2))

for compartment in image_set_loader.compartments:
    for unique_object in image_set_loader.unique_compartment_objects[compartment][1:2]:
        for channel1, channel2 in colocalization_channel_pairs:
            two_object_loader = ColocalizationTwoObject_Loader(
                image_set_loader=image_set_loader,
                image1=image_set_loader.image_set_dict[channel1],
                label_image1=image_set_loader.image_set_dict[compartment],
                object1=unique_object,
                image2=image_set_loader.image_set_dict[channel2],
                label_image2=image_set_loader.image_set_dict[compartment],
                object2=unique_object,
                compartment=compartment,
            )

            featurizer = ColocalizationFeaturization(
                image_set_loader=image_set_loader,
                two_object_loader=two_object_loader,
            )
            features = featurizer.process_features_for_output()
            organize_featurization_data(
                features=features,
                compartment=compartment,
                channel=f"{channel1}-{channel2}",
                label_index=unique_object,
                output=True,
                output_path=pathlib.Path(
                    f"../profiles_features/{image_set_loader.image_set_name}_colocalization_{compartment}_{channel1}_{channel2}_{unique_object}.parquet"
                ),
                image_set_name=image_set_loader.image_set_name,
                return_df=False,
            )

In [ ]:
end = time.time()
print(f"Time taken for {image_set_loader.image_set_name} featurization:", end - start)